In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


##**Load and Display image dimensions**

> This cell defines a function (get_nifti_dimensions) to retrieve the dimensions of NIfTI (Neuroimaging Informatics Technology Initiative) files using the nibabel library. It then iterates through image and label files, loading them and printing their respective dimensions.

In [ ]:
import nibabel as nib
import os

def get_nifti_dimensions(nifti_file):
    img = nib.load(nifti_file)
    return img.shape

image_folder = '/content/gdrive/MyDrive/segnetfinal/images1'
label_folder = '/content/gdrive/MyDrive/segnetfinal/labels1'

# Assuming filenames are consistent between images and labels
for image_file in os.listdir(image_folder):
    if image_file.endswith('_0000.nii.gz'):
        label_file = image_file.replace('_0000.nii.gz', '.nii.gz')

        image_path = os.path.join(image_folder, image_file)
        label_path = os.path.join(label_folder, label_file)

        image_dims = get_nifti_dimensions(image_path)
        label_dims = get_nifti_dimensions(label_path)

        print(f"Image: {image_file} - Dimensions: {image_dims}")
        print(f"Label: {label_file} - Dimensions: {label_dims}")


Image: 1.3.6.1.4.1.14519.5.2.1.6279.6001.108197895896446896160048741492_0000.nii.gz - Dimensions: (256, 256, 256)
Label: 1.3.6.1.4.1.14519.5.2.1.6279.6001.108197895896446896160048741492.nii.gz - Dimensions: (256, 256, 256)
Image: 1.3.6.1.4.1.14519.5.2.1.6279.6001.128023902651233986592378348912_0000.nii.gz - Dimensions: (256, 256, 256)
Label: 1.3.6.1.4.1.14519.5.2.1.6279.6001.128023902651233986592378348912.nii.gz - Dimensions: (256, 256, 256)
Image: 1.3.6.1.4.1.14519.5.2.1.6279.6001.278660284797073139172446973682_0000.nii.gz - Dimensions: (256, 256, 256)
Label: 1.3.6.1.4.1.14519.5.2.1.6279.6001.278660284797073139172446973682.nii.gz - Dimensions: (256, 256, 256)
Image: 1.3.6.1.4.1.14519.5.2.1.6279.6001.105756658031515062000744821260_0000.nii.gz - Dimensions: (256, 256, 256)
Label: 1.3.6.1.4.1.14519.5.2.1.6279.6001.105756658031515062000744821260.nii.gz - Dimensions: (256, 256, 256)
Image: 1.3.6.1.4.1.14519.5.2.1.6279.6001.122763913896761494371822656720_0000.nii.gz - Dimensions: (256, 256,

In [ ]:
pip install tensorflow

In [ ]:
pip install keras

In [ ]:
import tensorflow as tf
import keras
from keras import Model
from keras.layers import Input, Conv3D, MaxPooling3D, UpSampling3D, BatchNormalization
from keras.utils import to_categorical

print("TensorFlow version:", tf.__version__)
print("Keras version:", keras.__version__)

TensorFlow version: 2.15.0
Keras version: 2.15.0


##**Preprocess the image labels**

> This cell preprocesses images and labels by resizing them to dimensions (128x128x64) and saving them to new directories. It uses the nibabel library to load NIfTI files, performs resizing, and saves the resized data.

In [ ]:
import os
import nibabel as nib
import numpy as np

# Define the paths to the source image and label directories
image_dir = '/content/gdrive/MyDrive/segnetfinal/images1'
label_dir = '/content/gdrive/MyDrive/segnetfinal/labels1'

# Define the destination directories to save the resized data
output_image_dir = '/content/gdrive/MyDrive/segnetfinal/patchimg'
output_label_dir = '/content/gdrive/MyDrive/segnetfinal/patchlabel'

# Ensure the output directories exist
os.makedirs(output_image_dir, exist_ok=True)
os.makedirs(output_label_dir, exist_ok=True)

# Iterate through the files in the source image directory
for filename in os.listdir(image_dir):
    if filename.endswith('_0000.nii.gz'):
        # Load the image
        img_path = os.path.join(image_dir, filename)
        img_data = nib.load(img_path).get_fdata()

        # Resize the image to the desired dimensions (128x128x64)
        resized_img = img_data[:128, :128, :64]

        # Save the resized image to the output directory
        output_img_path = os.path.join(output_image_dir, filename)
        nib.save(nib.Nifti1Image(resized_img, np.eye(4)), output_img_path)

# Iterate through the files in the source label directory
for filename in os.listdir(label_dir):
    if filename.endswith('.nii.gz'):
        # Load the label
        label_path = os.path.join(label_dir, filename)
        label_data = nib.load(label_path).get_fdata()

        # Resize the label to the desired dimensions (128x128x64)
        resized_label = label_data[:128, :128, :64]

        # Save the resized label to the output directory
        output_label_path = os.path.join(output_label_dir, filename)
        nib.save(nib.Nifti1Image(resized_label, np.eye(4)), output_label_path)


##**Load and Preprocess Data for Training**

> This cell defines a function (load_data) to load and preprocess data for training a neural network.

> It loads images and labels, performs one-hot encoding on labels, and splits the data into training and validation sets using train_test_split from scikit-learn.

In [ ]:
import nibabel as nib
import numpy as np
import os
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.utils import to_categorical
from keras.models import Model
from keras.layers import Input, Conv3D, MaxPooling3D, UpSampling3D, BatchNormalization

# Function to load and preprocess data
def load_data(image_folder, label_folder):
    images = []
    labels = []

    for filename in os.listdir(image_folder):
        if filename.endswith('_0000.nii.gz'):
            img_path = os.path.join(image_folder, filename)
            label_path = os.path.join(label_folder, filename.replace('_0000.nii.gz', '.nii.gz'))

            img = nib.load(img_path).get_fdata()
            lbl = nib.load(label_path).get_fdata()

            images.append(img)
            labels.append(lbl)

    return np.array(images), np.array(labels)

# Paths to your image and label folders
image_folder = '/content/gdrive/MyDrive/segnetfinal/patchimg'
label_folder = '/content/gdrive/MyDrive/segnetfinal/patchlabel'

# Load and preprocess the data
images, labels = load_data(image_folder, label_folder)
labels = to_categorical(labels, num_classes=2)  # Convert labels to one-hot encoding

# Split data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)

In [ ]:
# Normalizing and reshaping the data
x_train = x_train[..., np.newaxis] / np.max(x_train)  # Add channel dimension and normalize
x_val = x_val[..., np.newaxis] / np.max(x_val)  # Add channel dimension and normalize

##**Defining Segnet Model Architecture**

> This cell defines the architecture of a 3D SegNet model using Keras. The model comprises an encoder-decoder structure for 3D image segmentation, with convolutional and pooling layers.

In [ ]:
from keras.layers import Conv3D, BatchNormalization, MaxPooling3D, UpSampling3D, Input, Conv3DTranspose
from keras.models import Model

# SegNet Model Definition for 128x128x64 input
def segnet_3d(input_shape=(128, 128, 64, 1), num_classes=2):
    inputs = Input(input_shape)

    # Encoder
    x = Conv3D(64, (3, 3, 3), activation='relu', padding='same')(inputs)
    x = BatchNormalization()(x)
    x = MaxPooling3D((2, 2, 2), padding='same')(x)

    # Decoder
    x = Conv3D(64, (3, 3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = UpSampling3D((2, 2, 2))(x)

    outputs = Conv3D(num_classes, (1, 1, 1), activation='softmax')(x)

    model = Model(inputs, outputs)
    return model


##**Train SegNet Model with Callbacks**

> This cell defines a custom callback (EpochTimeCallback) to print the time taken for each epoch during training. It also sets up two checkpoints: one to save the best model based on validation loss and another to save the model weights every 10 epochs.

> The SegNet model is then compiled and trained using the training and validation data, with the training progress and checkpoints monitored using the custom callbacks.

In [ ]:
import time
from keras.callbacks import Callback, ModelCheckpoint

# Custom callback to print epoch number and time taken
class EpochTimeCallback(Callback):
    def on_epoch_begin(self, epoch, logs=None):
        self.epoch_start_time = time.time()

    def on_epoch_end(self, epoch, logs=None):
        epoch_time = time.time() - self.epoch_start_time
        print(f"Epoch {epoch + 1}/{self.params['epochs']} - Time taken: {epoch_time:.2f} seconds")

# Define a checkpoint callback to save model weights
checkpoint_path = 'checkpoint_dir/'  # Set the directory to save checkpoints
os.makedirs(checkpoint_path, exist_ok=True)

# Save the best model checkpoint
best_checkpoint = ModelCheckpoint(
    os.path.join(checkpoint_path, 'best_checkpoint.h5'),
    monitor='val_loss',
    save_best_only=True,
    save_weights_only=True,
    verbose=1
)

# Save a checkpoint every 10 epochs
epoch_checkpoint = ModelCheckpoint(
    os.path.join(checkpoint_path, 'epoch_{epoch:02d}_checkpoint.h5'),
    period=10,
    save_weights_only=True,
    verbose=1
)

# Compile and train the model with the custom callback and checkpoints
model = segnet_3d()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Training the model with the custom callback and checkpoints
history = model.fit(
    x_train, y_train,
    validation_data=(x_val, y_val),
    batch_size=2,
    epochs=300,
    callbacks=[EpochTimeCallback(), best_checkpoint, epoch_checkpoint]  # Use the custom callback and checkpoints
)

# Evaluate the model
evaluation = model.evaluate(x_val, y_val)
print(f"Validation Loss: {evaluation[0]}, Validation Accuracy: {evaluation[1]}")


Epoch 1/300
 6/18 [=========>....................] - ETA: 5s - loss: 0.9445 - accuracy: 0.5203

18/18 [==============================] - ETA: 0s - loss: 0.6427 - accuracy: 0.7241Epoch 1/300 - Time taken: 11.65 seconds

Epoch 1: val_loss improved from inf to 0.54401, saving model to checkpoint_dir/best_checkpoint.h5
18/18 [==============================] - 12s 547ms/step - loss: 0.6427 - accuracy: 0.7241 - val_loss: 0.5440 - val_accuracy: 0.9656
Epoch 2/300
18/18 [==============================] - ETA: 0s - loss: 0.3874 - accuracy: 0.8768Epoch 2/300 - Time taken: 9.67 seconds

Epoch 2: val_loss improved from 0.54401 to 0.47708, saving model to checkpoint_dir/best_checkpoint.h5
18/18 [==============================] - 10s 543ms/step - loss: 0.3874 - accuracy: 0.8768 - val_loss: 0.4771 - val_accuracy: 0.9656
Epoch 3/300
18/18 [==============================] - ETA: 0s - loss: 0.3238 - accuracy: 0.9624Epoch 3/300 - Time taken: 9.20 seconds

Epoch 3: val_loss improved from 0.47708 to 0.43418, saving model to checkpoint_dir/best_checkpoint.h5
18/18 [==============================] - 9s